In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import pyxlsb
import xlsxwriter


#* INPUTS *#
i=29 #index of the first column where date is starting on weekly tab,before the weekly data, to drop from the Weekly DATA from Core WB.
n=4 # No. of new weeks in this Modeling Period.
M=135 # No. of Modeling Weeks from 7/3/2022
Core_WB = r"C:\Users\Dipankar.Mandal\OneDrive - Ipsos\MyPersonal\MMA-PMF_testing\Core_Workbook.xlsx"
Last_CWB = r"C:\Users\Dipankar.Mandal\OneDrive - Ipsos\MyPersonal\MMA-PMF_testing\CustomWorkbook1.xlsb"
today = datetime.today()
today = today.strftime("%m%d%y")
Pmf_File_Path = r"C:\Users\Sthitaprajna.Mishra\Downloads\pmf" + str(today)+ "_V1.xlsx"


NEW_WB=pd.read_excel(Core_WB,"Weekly")
NEW_WEEKLY = NEW_WB.drop(NEW_WB.columns[2:i],axis=1).iloc[:,:-3]
del NEW_WB

Framework=NEW_WEEKLY.iloc[:,:-(len(NEW_WEEKLY.columns)-2)]
Matchback_CWB=pd.read_excel(Last_CWB,engine='pyxlsb',sheet_name="Weekly",header=8).iloc[:,:(M-677)] 
Matchback_CWB = Matchback_CWB.drop(Matchback_CWB.columns[2:30],axis=1)
Matchback_CWB.rename(columns={'Variable Name':'Variable'},inplace=True)
Matchback_CWB.fillna(0,inplace=True)
Matchback_CWB  


: 

In [ ]:
## MatchBack TAb and Current Tab 
NEW_WEEKLY.columns=Matchback_CWB.columns
NEW_WEEKLY
Matchback_Weekly=pd.merge(Framework,Matchback_CWB,on=("ModelKey","Variable"),how='left')
del Matchback_CWB
Matchback_Weekly


In [ ]:
PMF= pd.concat([Framework,(Matchback_Weekly[Matchback_Weekly.columns[2:M+2]].div(NEW_WEEKLY[NEW_WEEKLY.columns[2:M+2]],fill_value=1))],axis=1)
PMF.fillna(1,inplace=True)
PMF.replace(np.inf,1,inplace=True)
PMF.iloc[:,len(PMF.columns)-n-1]


In [ ]:
## PMF TAB ##
PMF= pd.concat([Framework,(Matchback_Weekly[Matchback_Weekly.columns[2:M+2]].div(NEW_WEEKLY[NEW_WEEKLY.columns[2:M+2]],fill_value=1))],axis=1)
PMF.fillna(1,inplace=True)
PMF.replace(np.inf,1,inplace=True)
j=0
for j in range(n):
    PMF.iloc[:,len(PMF.columns)-n+j]=PMF.iloc[:,len(PMF.columns)-n-1]

# #Exceptions...
# PMF.loc[(PMF['Variable'] == "SPC_DGI") & (PMF['ModelKey'] == "New_Subs_Faas_FS_Bike"), PMF.columns[2:len(PMF.columns)]]= list(itertools.repeat(1,len(PMF.columns)-2))
# PMF.loc[(PMF['Variable'] == "SPC_DGI") & (PMF['ModelKey'] == "Existing_Subs_NonFaas_FS_Bike"), PMF.columns[2:len(PMF.columns)]]= list(itertools.repeat(1,len(PMF.columns)-2))
# # PMF.loc[(PMF['Variable'] == "TOR_SPD") & (PMF['ModelKey'] == "Row_Total"), PMF.columns[82:94]]= list(itertools.repeat(PMF[(PMF['ModelKey']=="Row_Total") &(PMF['Variable']=="TOR_SPD")].iloc[0,54],12))
# y=list(set(PMF["ModelKey"]))
# for i in y:
#     PMF.loc[(PMF['Variable'] == "RSC_PIM") & (PMF['ModelKey'] == i), PMF.columns[107:133]]= list(itertools.repeat(PMF[(PMF['ModelKey']==i) &(PMF['Variable']=="RSC_PIM")].iloc[0,106],26))
#     PMF.loc[(PMF['Variable'] == "SPC_PIM") & (PMF['ModelKey'] == i), PMF.columns[107:133]]= list(itertools.repeat(PMF[(PMF['ModelKey']==i) &(PMF['Variable']=="SPC_PIM")].iloc[0,106],26))
#     PMF.loc[(PMF['Variable'] == "STC_PIM") & (PMF['ModelKey'] == i), PMF.columns[107:133]]= list(itertools.repeat(PMF[(PMF['ModelKey']==i) &(PMF['Variable']=="STC_PIM")].iloc[0,106],26))
    


PMF[""]=""

Actual_Pred = ["Predicted","NNT_GUN","ENT_GUN","SBT_GUN","ROW_GUN","NFB_GUN","NNB_GUN","NNR_GUN","ENB_GUN","NBT_GUN","FSC_DGI","SBC_DGI","STC_DGI","TRC_DGI","SEC_IND"]
for i in Actual_Pred:
    PMF.loc[PMF['Variable']==i,PMF.columns[2:len(PMF.columns)-1]]= list(itertools.repeat(1,len(PMF.columns)-3))
PMF["0_Count"]=PMF.isin([0]).sum(axis=1)
PMF


In [ ]:
## Cross Check TAB ##
Cross_Check= pd.concat([Framework,(NEW_WEEKLY[NEW_WEEKLY.columns[2:M+2]].div(Matchback_Weekly[Matchback_Weekly.columns[2:M+2]],fill_value=1))],axis=1)
Cross_Check.fillna(1,inplace=True)
Cross_Check[""]=""
Cross_Check["0_Count"]=Cross_Check.isin([0]).sum(axis=1)
Cross_Check


In [ ]:
#for j in set(ADJ_Weekly['ModelKey']):
#    subset = ADJ_Weekly[(ADJ_Weekly['ModelKey'] == j) & ~ADJ_Weekly['Variable'].isin(["Predicted","NNT_GUN","ENT_GUN","SBT_GUN","ROW_GUN","NFB_GUN","NNB_GUN","NNR_GUN","ENB_GUN","NBT_GUN"])]
#    Predicted = subset.sum()
#    print(list(Predicted[2:129]))
#    ADJ_Weekly.loc[(ADJ_Weekly['ModelKey']==j) & (ADJ_Weekly['Variable']=="Predicted"),ADJ_Weekly.columns[2:129]]=list(Predicted[2:129])


In [ ]:
## NEW_WEEKLY TAB ##

ADJ_Weekly=pd.concat([Framework,(PMF[PMF.columns[2:M+2]].multiply(NEW_WEEKLY[NEW_WEEKLY.columns[2:M+2]]))],axis=1)

for j in set(ADJ_Weekly['ModelKey']):
    subset = ADJ_Weekly[(ADJ_Weekly['ModelKey'] == j) & ~ADJ_Weekly['Variable'].isin(["Predicted","NNT_GUN","ENT_GUN","SBT_GUN","ROW_GUN","NFB_GUN","NNB_GUN","NNR_GUN","ENB_GUN","NBT_GUN"])]
    Predicted = subset.sum()
    #print(list(Predicted[2:129]))#manual change
    ADJ_Weekly.loc[(ADJ_Weekly['ModelKey']==j) & (ADJ_Weekly['Variable']=="Predicted"),ADJ_Weekly.columns[2:137]]=list(Predicted[2:137])#manual change

#ADJ_Weekly_melted = ADJ_Weekly.melt(id_vars=['ModelKey', 'Variable'], var_name='Week', value_name='Value')
# Convert 'Value' column to numeric, coercing errors to NaN
#ADJ_Weekly_melted['Value'] = pd.to_numeric(ADJ_Weekly_melted['Value'], errors='coerce')
# Calculate the sum of non-'Predicted' values for each ModelKey
#sum_of_non_predicted = ADJ_Weekly_melted[ADJ_Weekly_melted['Variable'] != any(Actual_Pred)].groupby(['ModelKey','Week'])['Value'].sum().reset_index()
#print(sum_of_non_predicted)
# Update 'Predicted' values in the original DataFrame
#for index, row in ADJ_Weekly.iterrows():  #added this loop to iterate over rows
#    if row['Variable'] == 'Predicted':
#        for week_col in ADJ_Weekly.columns[2:]:  # Iterate through week columns
#            ADJ_Weekly.loc[index, week_col] = sum_of_non_predicted['Value'].loc[(sum_of_non_predicted['ModelKey'] == row['ModelKey']) & (sum_of_non_predicted['Week'] == week_col)].iloc[0]
            
ADJ_Weekly


In [ ]:
## EXPORT  ##
PMF_FRAMEWORK={
    "Matchback" :Matchback_Weekly,
    "Current"   :NEW_WEEKLY,
    "PMF"       :PMF,
    "Cross Check" :Cross_Check,
    "New_Weekly"  : ADJ_Weekly
}

with pd.ExcelWriter(Pmf_File_Path, engine='xlsxwriter') as writer:
    for sheet_name, ADJ_Weekly in PMF_FRAMEWORK.items():
        ADJ_Weekly.to_excel(writer, sheet_name=sheet_name, index=False)

print("PMF File Exported!")
